In [1]:
from onsager import crystal, supercell, cluster
import numpy as np
import pickle

In [2]:
# Load pre-saved arrays for 1nn
with open("supercellFCC.pkl", "rb") as fl:
    superFCC = pickle.load(fl)
crys = superFCC.crys
    
with open("jnetFCC.pkl", "rb") as fl:
    jnetFCC = pickle.load(fl)

a0 = 1.0
cut = 1.01*a0 # 2nn cutoff
jnetFCC_2nn = crys.jumpnetwork(0, cut)

In [4]:
# replace the 1nn jumpnetwork with what we had before
# for consistency
jnetFCC_2nn[0] = jnetFCC[0]

In [6]:
vacsite = cluster.ClusterSite((0, 0), np.zeros(3, dtype=int))
vacsiteInd = superFCC.index(np.zeros(3, dtype=int), (0, 0))[0]
assert vacsiteInd == 0

In [8]:
crys.lattice

array([[0. , 0.5, 0.5],
       [0.5, 0. , 0.5],
       [0.5, 0.5, 0. ]])

In [9]:
jnetFCC

[[((0, 0), array([ 0. , -0.5,  0.5])),
  ((0, 0), array([-0. ,  0.5, -0.5])),
  ((0, 0), array([ 0.5,  0. , -0.5])),
  ((0, 0), array([-0.5, -0. ,  0.5])),
  ((0, 0), array([ 0. , -0.5, -0.5])),
  ((0, 0), array([-0. ,  0.5,  0.5])),
  ((0, 0), array([-0.5, -0.5,  0. ])),
  ((0, 0), array([ 0.5,  0.5, -0. ])),
  ((0, 0), array([0.5, 0. , 0.5])),
  ((0, 0), array([-0.5, -0. , -0.5])),
  ((0, 0), array([ 0.5, -0.5,  0. ])),
  ((0, 0), array([-0.5,  0.5, -0. ]))]]

## State generation

In [10]:
RtoSiteInd = np.load("RtoSiteInd.npy")
SiteIndtoR = np.load("SiteIndtoR.npy")

In [11]:
N_units = superFCC.superlatt[0,0]
N_units

8

In [13]:
# Convert the jump displacements into lattice translation
dxR = []
for jList in jnetFCC_2nn:
    for (i, j), dx in jList:
        Rnn = np.dot(np.linalg.inv(crys.lattice), dx).astype(int)
        dxR.append(Rnn)
dxR = np.array(dxR)

In [14]:
N_ngb = len(dxR)
N_ngb

18

In [15]:
# Load the 1nn dxR and check they are in the same order
dxR_1nn = np.load("nnJumpLatVecs.npy")
assert np.array_equal(dxR_1nn, dxR[:dxR_1nn.shape[0]])

## Gather Nearest neighborhood

In [16]:
# Next, for each site, store the nearest neighbors
Nsites = len(superFCC.mobilepos)
print(Nsites)
NNList = np.zeros((N_ngb + 1, Nsites), dtype=int)
for siteInd in range(Nsites):
    NNList[0, siteInd] = siteInd
    Rsite = SiteIndtoR[siteInd]
    for ngbInd, Rnn in enumerate(dxR):
        RsiteNN = (Rsite + Rnn)%N_units
        siteIndNN = RtoSiteInd[RsiteNN[0], RsiteNN[1], RsiteNN[2]]
        NNList[ngbInd + 1, siteInd] = siteIndNN

512


In [17]:
NNList[:, 0]

array([  0,  15,  57, 449,  71, 448,  64,   7,   1,   8,  56, 456, 120,
       457, 127, 463, 121,  79, 505])

In [18]:
# Check that the 1nn neighbors don't have their orders changes
NNList_1nn = np.load("NNsites_sitewise.npy")
NNList_1nn[:, 0]

array([  0,  15,  57, 449,  71, 448,  64,   7,   1,   8,  56, 456, 120])

In [19]:
assert np.array_equal(NNList_1nn[:, :], NNList[:dxR_1nn.shape[0]+1, :])

## Store permutation of nearest neighbors

In [20]:
# For each group operation, store the permuation of the nearest neighbors
# First assign indices to nearest neighbor vectors
nntoIdx = {}
for ngbInd, Rnn in enumerate(dxR):
    RnnTup = tuple(Rnn)
    nntoIdx[RnnTup] = ngbInd+1

In [21]:
# Load the group operations with the same indices they were given
with open("GroupOpsIndices.pkl", "rb") as fl:
    GIndtoGDict = pickle.load(fl)

In [22]:
# Now for each group operation, store, the permutation of the nearest neighbors
GpermIdx = np.zeros((len(GIndtoGDict), N_ngb + 1), dtype=np.uint8)
for gInd, g in GIndtoGDict.items():
    for nnTup, nnIdx in nntoIdx.items():
        nnVec = np.array(nnTup)
        nnRot, _ = crys.g_pos(g.inv(), nnVec, (0, 0))
        nnIdxNew = nntoIdx[tuple(nnRot)]
        GpermIdx[gInd, nnIdx] = nnIdxNew

In [23]:
GpermIdx_1nn = np.load("GroupNNpermutations.npy")
assert np.array_equal(GpermIdx_1nn[:, :], GpermIdx[:, :dxR_1nn.shape[0]+1])

In [24]:
# Now save everything
np.save("NNsites_sitewise_2nn.npy", NNList)
np.save("nnJumpLatVecs_2nn.npy", dxR)
np.save("GroupNNpermutations_2nn.npy", GpermIdx)